# ERA5 correction 
Adapted from Eric Gagliano's notebook here: https://github.com/gbrencher/AMATH563-InSAR-denoiser/blob/main/data-processing/era5_correction_individual_igrams.ipynb

Note: This code will only work if in the MintPy source code (tropo_pyaps3.py) we comment out lines 657 (ref_y, ref_x = int(atr['REF_Y']), int(atr['REF_X'])) and 666 (data -= data[ref_y, ref_x]). This code subtracts out the value at the reference point. Since we haven't set a reference point in the metadata of these interferograms, it won't work. We can handle this later

In [1]:
import mintpy
from pathlib import Path
from dateutil.parser import parse as parse_date
from pathlib import Path
from typing import List, Union
from osgeo import gdal
from pathlib import Path
from typing import List, Union
import os
import glob
import matplotlib.pyplot as plt
from mintpy.utils import ptime, readfile, writefile, utils as ut

In [2]:
orbit_list = ['AT151', 'DT56']
year_list = ['2017', '2018', '2019', '2020', '2021', '2022']
frame_list = ['frame_1', 'frame_2', 'frame_3']

In [3]:
# Function to write to MintPy config file
def write_config_file(out_file, CONFIG_TXT, mode='a'): 
    """Write configuration files for MintPy to process products"""
    if not os.path.isfile(out_file) or mode == 'w':
        with open(out_file, "w") as fid:
            fid.write(CONFIG_TXT)
        print('write configuration to file: {}'.format(out_file))
    else:
        with open(out_file, "a") as fid:
            fid.write("\n" + CONFIG_TXT)
        print('add the following to file: \n{}'.format(CONFIG_TXT))

In [4]:
def correct_era5(orbit_list, frame_list, year_list, weather_dir = '/tmp'):
    home_path_d = '/mnt/d/indennt'
    home_path = '/mnt/c/Users/qbren/Desktop/taco/projects/indennt/proc/data'
    for orbit in orbit_list:
        for frame in frame_list:
            for year in year_list:
                print(f'working on {orbit}, {frame}, {year}')
                data_path = f'{home_path_d}/hyp3/{orbit}/{frame}/{year}'
                hyp3_list = os.listdir(data_path)
                for i, granule in enumerate(hyp3_list):
                    print(f'working on {i}/{len(hyp3_list)}')
                    granule_path = f'{data_path}/{granule}'
                    mintpy_path = f'{home_path}/signal_mintpy/{orbit}/{frame}/{granule}'

                    if os.path.exists(f'{granule_path}/{granule}_unw_phase_ERA5.tif'):
                        print(f'ERA5 already in {granule}, skipping') 
                        continue

                    # make output dir for mintpy
                    if not os.path.exists(mintpy_path):
                        os.mkdir(mintpy_path)
                    
                    os.chdir(mintpy_path)
    
                    CONFIG_TXT=f"""
                    mintpy.load.processor        = hyp3
                    ##---------interferogram datasets:
                    mintpy.load.unwFile          = {granule_path}/*_unw_phase.tif
                    mintpy.load.corFile          = {granule_path}/*_corr.tif
                    ##---------geometry datasets:
                    mintpy.load.demFile          = {granule_path}/*_dem.tif
                    mintpy.load.incAngleFile     = {granule_path}/*_lv_theta.tif
                    mintpy.load.azAngleFile      = {granule_path}/*_lv_phi.tif
                    mintpy.load.waterMaskFile    = {granule_path}/*_water_mask.tif
                    """
                    mintpy_config = f'{mintpy_path}/{frame}_{year}_Sen{orbit}.txt'
                    write_config_file(mintpy_config, CONFIG_TXT)
    
                    !smallbaselineApp.py {mintpy_config} --dostep load_data
                    geom_path = 'inputs/geometryGeo.h5'
                
                    unw_filename = f'{granule}_unw_phase.unw'
                
                    data, atr = readfile.read(f'{granule_path}/{granule}_unw_phase.tif', datasetName='phase')
                    atr['FILE_TYPE'] = '.unw'
                    writefile.write({'phase':data},unw_filename,metadata=atr)
                    
                    !tropo_pyaps3.py -f $unw_filename -g $geom_path --weather-dir $weather_dir
                    unw_filename_corr = f'{unw_filename[:-4]}_ERA5.unw'
                    unw_filename_out = f'{unw_filename[:-4]}_ERA5.tif'
                    !gdal_translate -of GTiff -b 2 -a_srs EPSG:32613 $unw_filename_corr $unw_filename_out
                    !cp *_ERA5.tif $granule_path && rm -R $mintpy_path

In [5]:
correct_era5(orbit_list, frame_list, year_list)

working on AT151, frame_1, 2017
working on 0/26
ERA5 already in S1BB_20170608T010147_20170702T010148_VVP024_INT40_G_ueF_F55B, skipping
working on 1/26
ERA5 already in S1BB_20170608T010147_20170714T010149_VVP036_INT40_G_ueF_01C4, skipping
working on 2/26
ERA5 already in S1BB_20170608T010147_20170726T010149_VVP048_INT40_G_ueF_30BA, skipping
working on 3/26
ERA5 already in S1BB_20170608T010147_20170807T010150_VVP060_INT40_G_ueF_EE72, skipping
working on 4/26
ERA5 already in S1BB_20170702T010148_20170714T010149_VVP012_INT40_G_ueF_63A6, skipping
working on 5/26
ERA5 already in S1BB_20170702T010148_20170726T010149_VVP024_INT40_G_ueF_DAC3, skipping
working on 6/26
ERA5 already in S1BB_20170702T010148_20170807T010150_VVP036_INT40_G_ueF_73DF, skipping
working on 7/26
ERA5 already in S1BB_20170702T010148_20170819T010151_VVP048_INT40_G_ueF_5DFA, skipping
working on 8/26
ERA5 already in S1BB_20170714T010149_20170726T010149_VVP012_INT40_G_ueF_2892, skipping
working on 9/26
ERA5 already in S1BB_2017

/home/gbrench/sw/miniconda3/envs/mintpy/lib/python3.11/site-packages/osgeo/gdal.py:287: FutureWarning: Neither gdal.UseExceptions() nor gdal.DontUseExceptions() has been explicitly called. In GDAL 4.0, exceptions will be enabled by default.
  warnings.warn(


write file: S1AA_20190808T131007_20190901T131008_VVP024_INT40_G_ueF_70C9_unw_phase.unw
write file: S1AA_20190808T131007_20190901T131008_VVP024_INT40_G_ueF_70C9_unw_phase.unw.rsc
weather model: ERA5 - dry (hydrostatic) and wet delay
weather directory: /tmp
output tropospheric delay     time-series file: inputs/ERA5.h5
output corrected displacement time-series file: S1AA_20190808T131007_20190901T131008_VVP024_INT40_G_ueF_70C9_unw_phase_ERA5.unw
read dates/time info from file: S1AA_20190808T131007_20190901T131008_VVP024_INT40_G_ueF_70C9_unw_phase.unw
time of cloest available product: 13:00 UTC
/home/gbrench/sw/miniconda3/envs/mintpy/lib/python3.11/site-packages/osgeo/gdal.py:287: FutureWarning: Neither gdal.UseExceptions() nor gdal.DontUseExceptions() has been explicitly called. In GDAL 4.0, exceptions will be enabled by default.
  warnings.warn(

--------------------------------------------------------------------------------
Download global atmospheric model files...
update mode: ON
out

Exception ignored in atexit callback: <bound method InteractiveShell.atexit_operations of <ipykernel.zmqshell.ZMQInteractiveShell object at 0x7f89e6bdf650>>
Traceback (most recent call last):
  File "/home/gbrench/sw/miniconda3/envs/mintpy/lib/python3.11/site-packages/IPython/core/interactiveshell.py", line 3875, in atexit_operations
    self._atexit_once()
  File "/home/gbrench/sw/miniconda3/envs/mintpy/lib/python3.11/site-packages/IPython/core/interactiveshell.py", line 3854, in _atexit_once
    self.reset(new_session=False)
  File "/home/gbrench/sw/miniconda3/envs/mintpy/lib/python3.11/site-packages/IPython/core/interactiveshell.py", line 1373, in reset
    self.history_manager.reset(new_session)
  File "/home/gbrench/sw/miniconda3/envs/mintpy/lib/python3.11/site-packages/IPython/core/history.py", line 597, in reset
    self.dir_hist[:] = [Path.cwd()]
                        ^^^^^^^^^^
  File "/home/gbrench/sw/miniconda3/envs/mintpy/lib/python3.11/pathlib.py", line 907, in cwd
    r

## set reference point
switch to env with xarray

In [1]:
import os
import numpy as np
import pandas as pd
import rasterio as rio
import matplotlib.pyplot as plt
import xarray as xr
import datetime as dt
import rioxarray
import seaborn as sns
import geopandas as gpd
from glob import glob

In [2]:
# load in single igram and other data 
def hyp3_to_xarray_single(path):
    '''
    Reads hyp3 outputs into xarray dataset from single hyp3 folder 
    '''
    # globs for data to load
    unw_phase_path = glob(f'{path}/*unw_phase.tif')[0]
    era5_path = glob(f'{path}/*ERA5.tif')[0]
    meta_path = glob(f'{path}/S1*.txt')[0]

    # list granules for coordinate
    granule = os.path.split(unw_phase_path)[-1][0:-14]

    d = {}
    with open(meta_path) as f:
        for line in f:
            (key, val) = line.split(':')
            d[key] = str.strip(val)

    # read unw_phase into data array and assign coordinates
    da = xr.open_dataset(unw_phase_path)
    da = da.assign_coords({'granule':('granule', [granule])})
    for item in d.keys():
            da = da.assign_coords({item:('granule', [d[item]])})
    
    # concatenate into dataset and rename variable
    ds = da.rename({'band_data': 'unw_phase'})

    #open coherence and dem into datasets
    era5_ds = xr.open_dataset(era5_path)

    # add coherence and dem to unw_phase dataset
    ds['era5_phase'] = (('band', 'y', 'x'), era5_ds.band_data.values)

    # remove band coordinate
    ds = ds.squeeze()

    return ds

In [3]:
def set_era5_ref(orbit, frame_list, year_list):
    home_path = '/mnt/d/indennt'
    for orbit in orbit_list:
        for frame in frame_list:
            for year in year_list:
                data_path = f'{home_path}/hyp3/{orbit}/{frame}/{year}'
                hyp3_list = os.listdir(data_path)
                for i, granule in enumerate(hyp3_list):
                    print(f'working on {orbit}, {frame}, {year}, {i}/{len(hyp3_list)}')
                    granule_path = f'{data_path}/{granule}'
                    os.chdir(granule_path)
                    
                    # if readme files are still present
                    #!rm *.md*
    
                    ds = hyp3_to_xarray_single(granule_path)
                    ref_value = ds.era5_phase.sel(y=ds['Y coordinate of the reference point in the map projection'],
                                  x=ds['X coordinate of the reference point in the map projection'],
                                  method='nearest').item()
                    ds['era5_phase'] = ds['era5_phase'] - ref_value
                    ds.era5_phase.rio.to_raster(f'{granule_path}/{granule}_unw_phase_ERA5.tif')

In [6]:
orbit_list = ['AT151', 'DT56']
year_list = ['2017', '2018', '2019', '2020', '2021', '2022']
frame_list = ['frame_1', 'frame_2', 'frame_3']

In [7]:
set_era5_ref(orbit_list, frame_list, year_list)

working on AT151, frame_1, 2017, 0/26
working on AT151, frame_1, 2017, 1/26
working on AT151, frame_1, 2017, 2/26
working on AT151, frame_1, 2017, 3/26
working on AT151, frame_1, 2017, 4/26
working on AT151, frame_1, 2017, 5/26
working on AT151, frame_1, 2017, 6/26
working on AT151, frame_1, 2017, 7/26
working on AT151, frame_1, 2017, 8/26
working on AT151, frame_1, 2017, 9/26
working on AT151, frame_1, 2017, 10/26
working on AT151, frame_1, 2017, 11/26
working on AT151, frame_1, 2017, 12/26
working on AT151, frame_1, 2017, 13/26
working on AT151, frame_1, 2017, 14/26
working on AT151, frame_1, 2017, 15/26
working on AT151, frame_1, 2017, 16/26
working on AT151, frame_1, 2017, 17/26
working on AT151, frame_1, 2017, 18/26
working on AT151, frame_1, 2017, 19/26
working on AT151, frame_1, 2017, 20/26
working on AT151, frame_1, 2017, 21/26
working on AT151, frame_1, 2017, 22/26
working on AT151, frame_1, 2017, 23/26
working on AT151, frame_1, 2017, 24/26
working on AT151, frame_1, 2017, 25